In [44]:
from datetime import datetime
from string_matcher import ngrams,awesome_cossim_top,get_matches_df
import pandas as pd, numpy as np, re, sparse_dot_topn.sparse_dot_topn as ct
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from math import cos, asin, sqrt

def convert_to_sqm(i):
    return i/10.764

def clean_price(i):
    f_string = ''
    for char in i:
        if char.isnumeric():
            f_string += char
    return float(f_string)

def clean_sqm(i):
    i = i.replace(',','')
    return float(i)

def clean_address(s):
    s = s.replace('AVENUE','AVE')
    s = s.replace('STREET','ST')
    s = s.replace('ROAD','RD')
    s = s.replace('BLOCK','BLK')
    s = s.replace('STREET','ST')
    return s

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) 

def closest(data, v):
    t = min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))
    return distance(t['lat'],t['lon'],v['lat'],v['lon'])

class mapping():
    def __init__(self,geocode=True,include_buildings=True):
        self.geocode = True
        self.include_buildings = True

    def add_buildings(self,df):
        buildings = pd.read_json('singapore-postal-codes-master/buildings.json')
        buildings.BUILDING = buildings.BUILDING.str.lower()
        buildings.drop_duplicates('BUILDING',inplace=True)
        places = {'primary_school':'primary school',
                        'seconday_school':'secondary school|high school',
                        'junior_college':'junior college',
                        'polytechnic':'polytechnic',
                        'mrt_lrt':'mrt|lrt',
                        'preschool':'preschool'}
        df.reset_index(inplace=True,drop=True)
        for place in places:
            search_key = places[place]
            temp_df = buildings[buildings['BUILDING'].str.contains(search_key)][['LATITUDE','LONGTITUDE']]
            temp_df.reset_index(inplace=True,drop=True)
            places_lon_lat = []
            for i in range(len(temp_df)):
                temp_dict = {'lat':temp_df.loc[i,'LATITUDE'],'lon':temp_df.loc[i,'LONGTITUDE']}
                places_lon_lat.append(temp_dict)
            
            df['distance_to_'+str(place)] = [closest(places_lon_lat,{'lat':df.loc[idx,'LAT'],'lon':df.loc[idx,'LON']}) for idx in range(len(df))]
        return df

    def transform(self,df,website):
        self.df = df
        if website == 'guru':
            df['price'] = df['price'].apply(clean_price)
            df['floor_area'] = df['floor_area'].apply(convert_to_sqm)
            df['address'] = df['address'].str.upper()
            df['address'] = df['address'].apply(clean_address)
            df.dropna(inplace=True)
            df.reset_index(inplace=True,drop=True)
            df['year_built'] = df['year_built'].astype(int)
            df['remaining_lease'] = 99 - (2020 - df['year_built'])
            df.drop(['list_date','year_built','date_scraped'],axis=1,inplace=True)
            df.rename({'bedrooms':'No of bedrooms','bathrooms':'No of bathrooms','floor_area':'floor_area_sqm'},inplace=True,axis=1)

        elif website == '99co':
            df['price'] = df['price'].apply(clean_price)
            df['address'] = df['address'].str.upper()
            df['address'] = df['address'].apply(clean_address)
            df.dropna(inplace=True)
            df.reset_index(inplace=True,drop=True)
            df['year_built'] = df['year_built'].astype(int)
            df['remaining_lease'] = 99 - (2020 - df['year_built'])
            df.drop(['year_built','date_scraped'],axis=1,inplace=True)
            df.rename({'bedrooms':'No of bedrooms','bathrooms':'No of bathrooms','floor_area':'floor_area_sqm'},inplace=True,axis=1)
    
        if self.geocode:
            vectorizer = TfidfVectorizer(min_df=1,analyzer=ngrams)
            geocode = pd.read_csv('hdb resale geocoded.csv')
            geocode = geocode[['Address','LAT','LON']]
            tf_idf_matrix_clean = vectorizer.fit_transform(geocode['Address'])
            tf_idf_matrix_dirty = vectorizer.transform(df['address'])

            matches = awesome_cossim_top(tf_idf_matrix_dirty,tf_idf_matrix_clean.transpose(),1,0)
            matches_df = get_matches_df(matches,df['address'],geocode['Address'],top=0)

            df = pd.merge(df,matches_df[['address','Address']],how='left',on='address')
            df.drop_duplicates(inplace=True)
            df = pd.merge(df,geocode,how='left',on='Address')
            df.drop_duplicates(inplace=True)
            df['floor_area_sqm'] = df['floor_area_sqm'].apply(clean_sqm)

        if self.include_buildings:
            df = self.add_buildings(df)

        df = df[['No of bedrooms', 'No of bathrooms', 'floor_area_sqm',
        'remaining_lease', 'LAT', 'LON', 'distance_to_primary_school',
        'distance_to_seconday_school', 'distance_to_junior_college',
        'distance_to_polytechnic', 'distance_to_mrt_lrt', 'distance_to_preschool','price', 'link']]
        df.dropna(inplace=True)
        return df

In [27]:
df = pd.read_csv('property_guru.csv')

In [29]:
guru_reformatted = mapping().transform(df,website='guru')

In [35]:
guru_reformatted.to_csv('guru_reformatted.csv',index=False)

In [45]:
co = pd.read_csv('99co.csv')

In [46]:
co_reformatted = mapping().transform(co,website='99co')

In [47]:
co_reformatted.to_csv('99co_reformatted.csv',index=False)